In [1]:
import os
os.chdir('../')

In [2]:
pwd

'c:\\Users\\asbpi\\Desktop\\ASB\\Data_Science\\Projects\\MY Projects\\Stroke-Risk-Prediction'

In [3]:
# Entity

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    train_data_path: Path
    model_name: str
    n_estimators: int
    criterion: str
    max_depth: int
    min_samples_split: int
    min_samples_leaf: int
    target_column: str

In [5]:
# Configuration Manager

In [6]:
from stroke_risk.constants import *
from stroke_risk.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_training_config(self) -> ModelTrainingConfig:
        config = self.config.model_training
        params = self.params.RandomForestClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_training_config = ModelTrainingConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            model_name = config.model_name,
            n_estimators = params.n_estimators,
            criterion = params.criterion,
            max_depth = params.max_depth,
            min_samples_split = params.min_samples_split,
            min_samples_leaf = params.min_samples_leaf,
            target_column = schema.name
        )

        return model_training_config

In [8]:
# Components

In [9]:
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
from imblearn.combine import SMOTETomek
from sklearn.ensemble import RandomForestClassifier
import joblib
from stroke_risk import logger

In [10]:
class ModelTraining:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config

    
    def training(self):
        train_data = pd.read_csv(self.config.train_data_path)

        x_train = train_data.drop([self.config.target_column], axis=1)
        y_train = train_data[self.config.target_column]

        scaler = StandardScaler()
        x_train = scaler.fit_transform(x_train)

        imb = SMOTETomek(sampling_strategy='minority', random_state=42)
        x_train , y_train = imb.fit_resample(x_train,y_train)

        model = RandomForestClassifier(
            n_estimators = self.config.n_estimators,
            criterion = self.config.criterion,
            max_depth=self.config.max_depth,
            min_samples_split = self.config.min_samples_split,
            min_samples_leaf = self.config.min_samples_leaf,
            max_features='sqrt',
            bootstrap=True,
            random_state=42
            )
        
        model.fit(x_train, y_train)

        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))
        logger.info('Model Trained')

In [11]:
# Pipeline

In [12]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_training_config()
    model_trainer_config = ModelTraining(config=model_trainer_config)
    model_trainer_config.training()
except Exception as e:
    raise e

[2023-12-27 00:14:18,619: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-27 00:14:18,628: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-27 00:14:18,637: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-27 00:14:18,638: INFO: common: created directory at: artifacts]
[2023-12-27 00:14:18,642: INFO: common: created directory at: artifacts/model_training]
[2023-12-27 00:14:19,081: INFO: 2572249179: Model Trained]
